# 1. CNN

## 1.1 Operator Implementation by Numpy

### 1.1.1 Conv

In [1]:
import numpy as np

def GussianKernel(kernel_size):
    guassian = lambda x: 1/np.sqrt(2*np.pi)*np.exp(-1/2*x)
    kernel = np.zeros((kernel_size, kernel_size))
    for i in range(kernel_size):
        for j in range(kernel_size):
            kernel[i, j] = guassian(
                (kernel_size/2 - i - 1/2)**2 + (kernel_size/2-j - 1/2)**2)
    return kernel


def Conv(inp, stride=1, kernel_size=1):
    kernel = GussianKernel(kernel_size=1)
    pass

### 1.1.2 Pooling

In [ ]:
import numpy as np

def Pooling(input: np.array, stride: int = 2):
    (B, C, H, W) = input.shape
    output  = np.zeros((B, C, H//stride, W//stride))
    for b in range(B):
        for c in range(C):
            for h in range(H//stride):
                for w in range(W//stride):
                    region = input[b, 
                                   c, 
                                   h*stride: h*stride + stride,
                                   w*stride: w*stride + stride].reshape(-1)
                    output[b, c, h, w] = np.max(region)
    return output
            

## 1.2 Object Detection

In [2]:
import numpy as np

def box_iou(a, b):
    """
    Args:
        boxes1 (array[N, 4]): first set of boxes
        boxes2 (array[M, 4]): second set of boxes
    Returns:
        array[N, M]: the NxM matrix containing the IoU values
    """
    area = (b[:, 2] - b[:, 0]) * (b[:, 3] - b[:, 1])
    iw = np.minimum(np.expand_dims(a[:, 2], axis=1), b[:, 2]) - np.maximum(
        np.expand_dims(a[:, 0], 1), b[:, 0])
    ih = np.minimum(np.expand_dims(a[:, 3], axis=1), b[:, 3]) - np.maximum(
        np.expand_dims(a[:, 1], 1), b[:, 1])
    iw = np.maximum(iw, 0)
    ih = np.maximum(ih, 0)
    ua = np.expand_dims(
        (a[:, 2] - a[:, 0]) * (a[:, 3] - a[:, 1]), axis=1) + area - iw * ih
    ua = np.maximum(ua, np.finfo(float).eps)
    intersection = iw * ih
    return intersection / ua

a = np.array([
    [1, 2, 3, 4],
    [5, 6, 7, 7]
    ])
print(np.expand_dims(a[:, 3], axis=1))

[[4]
 [7]]


In [ ]:
def NMS(dets, thresh):
    x1 = dets[:, 0]
    y1 = dets[:, 1]
    x2 = dets[:, 2]
    y2 = dets[:, 3]
    scores = dets[:, 4]
    areas = (x2 - x1 + 1) * (y2 - y1 + 1)
    order = scores.argsort()[::-1]

	temp = []
    while order.size > 0:
        i = order[0]
        temp.append(i)
        xx1 = np.maximum(x1[i], x1[order[1:]])
        yy1 = np.minimum(y1[i], y1[order[1:]])
        xx2 = np.minimum(x2[i], x2[order[1:]])
        yy2 = np.maximum(y2[i], y2[order[1:]])

        w = np.maximum(0.0, xx2 - xx1 + 1)
        h = np.maximum(0.0, yy2 - yy1 + 1)
        inter = w * h
        ovr = inter / (areas[i] + areas[order[1:]] - inter)

        inds = np.where(ovr <= thresh)[0]
			order = order[inds + 1]
    return temp


dets = np.array([[310, 30, 420, 5, 0.6],
                    [20, 20, 240, 210, 1],
                    [70, 50, 260, 220, 0.8],
                    [400, 280, 560, 360, 0.7]])
thresh = 0.4
keep_dets = NMS(dets, thresh)
print(keep_dets)
print(dets[keep_dets])